In [ ]:

import numpy as np
import torch
import cv2 as cv
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torch.optim.lr_scheduler import StepLR
from data_loaders import CAERSDataset
from utils import get_transform, accuracy, accuracy_julia
from model import model_generator
from tqdm import tqdm
import os.path as osp
from keras.models import load_model
import tensorflow as tf
import os


In [ ]:

captura = cv.VideoCapture(0)
model_path = '.\models\\faces_19_95.01295733262594.pth'
model = torch.load(model_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Se a maquina não tiver GPU o teste será rodado na CPU


model.to(device)

opt = RMSprop(model.parameters(), lr=3e-3)
lr_scheduler = StepLR(opt, 60, 0.4)





In [ ]:
import matplotlib.pyplot as plt
ret, frame = captura.read()
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
faces = face_cascade.detectMultiScale(gray, 1.1, 2)
plt.imshow(frame)

In [ ]:
x,y,w,h = faces[0]
print(x, y, w, h)

In [ ]:
frame_ = frame[y:y+h, x:x+w, :] # y+ h e x+w
plt.imshow(frame_)

In [ ]:
type(frame_)
frame__ = torch.asarray(frame_)
print(type(frame__))

In [ ]:
print(frame_.shape, frame__.shape, torch.unsqueeze(frame__, dim=0).shape)

In [ ]:
preds = model(frame__.to(device))
preds = preds.to('cpu')

In [ ]:
preds = model(torch.unsqueeze(frame__, dim=0))

In [ ]:
#tf.disable_v2_behavior()
#(x,y),(x+w,y+h)

while(1):

    ret, frame = captura.read()
    #cv.imshow("Video", frame)
    # convert to grayscale of each frames
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # read haacascade to detect faces in input image
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

        # detects faces in the input image
    faces = face_cascade.detectMultiScale(gray, 1.1, 2)

    #face_tensor = tf.convert_to_tensor(faces)
    #face_tensor = np.load(faces.decode())
    

    print('Number of detected faces:', len(faces))\

        # loop over all the detected faces
    for (x,y,w,h) in faces:
        ret, frame = captura.read()
        #face = tf.convert_to_tensor(faces, dtype=None, dtype_hint=None, name=None)
        #preds = model(frame.to(device))
        #preds = preds.to('cpu')
          
        # To draw a rectangle around the detected face  
        
        face_cut =  frame[y:y+h, x: x+w]
        preds = model(torch.unsqueeze(face_cut, dim=0))
        preds = model(face_cut.to(device))
        preds = preds.to('cpu')
        image= cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        cv.putText(image,preds, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Display an image in a window
        cv.imshow('emotionRAM',image)

        k = cv.waitKey(30) & 0xff
        if k == 27:
           break

captura.release()
cv.destroyAllWindows()